This endpoint extends the usual symcat-synthea definition by allowing non-symptoms to show up in a condition's symptom list.

A non-symptom referes to a known symptom which already does not belong to a condition's set of known symptoms.

Including these symptoms - albeit - with a lower probability (say < 5%) increases the robustness of the trained models to *"noise"* and would help improve generalization.

We'll start first without the nlice questions

In [ ]:
DATA_DIR = "/Users/teliov/TUD/Thesis/Medvice/Notebooks/data/"

In [ ]:
nlice_plus_definition = "/Users/teliov/TUD/symcat-to-synthea/output/ai-med-plus.json"

In [ ]:
import json

In [ ]:
with open(nlice_plus_definition) as fp:
    nlice_plus_data = json.load(fp)

In [ ]:
# first extract the list of symptoms
symptoms = set([])

In [6]:
min_probability = 100
for cond_name, data in nlice_plus_data.items():
    for _symptom in data["symptoms"].keys():
        symptoms.add(_symptom)
        _symptom_prob = data["symptoms"][_symptom].get("probability")
        
        if _symptom_prob < min_probability and _symptom_prob > 1:
            min_probability = _symptom_prob

In [7]:
# now for each condition which does not have a symptom, that symptom is added with a probability which is less than
# the least known symptom probability 
print(min_probability)

2


In [8]:
# we'll update our data files
for cond_name in nlice_plus_data:
    condition = nlice_plus_data.get(cond_name)
    condition_symptoms = condition.get("symptoms")
    
    for _symptom in symptoms:
        if _symptom in condition_symptoms:
            continue
        condition_symptoms[_symptom] = {
            "slug": _symptom,
            "probability": 1,
            "nlice": {}
        }
    condition["symptoms"] = condition_symptoms

In [9]:
extended_nlice_file = "/Users/teliov/TUD/symcat-to-synthea/output/ai-med-extended.json"
with open(extended_nlice_file, "w") as fp:
    json.dump(nlice_plus_data, fp, indent=4)